# DSCI 512 Lecture 7

In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict
import urllib.request

In [2]:
import pulp

# To install pulp, run `pip install pulp`

Outline:

- Discrete optimization: intro/example (20 min)
- Constraints (5 min)
- TA assignment example: introduction (5 min)
- (Time permitting) Let's try solving this by hand (10 min)
- Break (5 min)
- Formulating an optimization problem (35 min)

No time for:

- Types of discrete optimization problems (5 min)
- (Time permitting) T/F questions (10 min)

## Discrete optimization intro/example (20 min)

- _Optimization_ refers to maximizing or minimizing a function with respect to its inputs.
  - For example, find the minimum of $f(x)=(x-0.9)^2$. The answer is $0$, when $x=0.9$.
- _Discrete optimization_ refers to optimization where the variables are _discrete_ (as opposed to continuous).
  - For example, find the minimum of $f(x)=(x-0.9)^2$ where $x$ is an integer. The answer is $0.01$, when $x=1$.




#### Example problem

Another example: find the length-8 string of letters (e.g. `aaaaaaaa`, `abcdefgh`, etc.) that contains the most English words in it.

- Because the problem is discrete, we can count the number of "options" or "configurations".
  - In the first case above, $x$ is any integer so there are infinitely many.
  - In this example, if we're just using the 26 lower-case letters, we have $26^8$ possibilities.

In [3]:
26**8

208827064576

- That is a lot!


If the number of possibilities is finite, you can enumerate (or list) all possible configurations (inputs).
  - In our example, there is `aaaaaaaa`, `aaaaaaab`, `aaaaaaac`, ..., `zzzzzzzz`. 
  - This is called "brute force search".
  - Often, it's not computationally feasible to search through all possible combinations because there are too many. 
  - So we need smarter algorithms that can somehow learn to ignore or "throw away" large numbers of possibilities without explicitly checking them.

#### Specifying an optimization problem

To specify the problem, we need to specify a few things:

1. A specification of the space of possible inputs.
2. An _objective function_ which takes an input and computes a score. 
3. Are we maximizing or minimizing?

In the above, the space of possible inputs is all length-8 strings. And the objective function is the number of English words in the string, and we are maximizing it. 

#### Solving an optimization problem with brute force (example)

- First, let's implement the objective function in Python

In [4]:
words_file = urllib.request.urlopen("https://gist.githubusercontent.com/deekayen/4148741/raw/01c6252ccc5b5fb307c1bb899c95989a8a284616/1-1000.txt")
word_list = {s.decode("utf-8").strip().lower() for s in words_file.readlines()}

- Note on the above: we're just using the 1000 most common English words. 
- I tried a full corpus of English words but it turns out a lot of strange things are English words, like "t" or "ea".

In [7]:
def english_words_in_string(s):
    """
    Find all the English words within a string.
    
    Parameters
    ----------
    s : str
        The input string.
       
    Returns
    -------
    set
        The set of English words contained in s.
        
    Example
    -------
    >>> english_words_in_string("theapple")
    {'a', 'apple', 'e', 'ea', 'h', 'he', 'heap', 'l', 'p', 't', 'th', 'the'}    
    """
    
    words = set()
    for start in range(len(s)):
        for end in range(start+1, len(s)+1):
            if s[start:end] in word_list:
                words.add(s[start:end])
    return words

In [8]:
english_words_in_string("theapple")

{'a', 'apple', 'he', 'the'}

It looks like the length-8 string `theapple` contains a bunch of words.

Next, let's implement the objective function:

In [9]:
def objective_fun(s):
    """Count the number of English words contained within a string."""
    return len(english_words_in_string(s))

objective_fun("theapple")

4

The objective function for `theapple` is 4. We are trying to find the string (or strings - the solution is not always unique) that make this value as large as possible.

**Note:** For the next parts, we'll restrict ourselves to only using the first 5 letters of the alphabet (a, b, c, d, e) so that the code isn't too slow. This way we should only have

In [11]:
5**8

390625

possibilities, which is reasonable. We'll grab this function from lab 2's optional question on weird recursive code:

In [12]:
def get_inputs(letters, n):
    """
    Return all length-n strings using letters.
    
    Parameters
    ----------
    letters : list
        The list of letters (could be a string or a list)
    
    n : int
        The desired length.
        
    Returns
    -------
    list
        A list of the resulting strings.
        
    Examples
    --------
    get_inputs(['a', 'b', 'c'], 2)
    ['aa', 'ab', 'ac', 'ba', 'bb', 'bc', 'ca', 'cb', 'cc']
    """
    
    if n == 0:
        return [""]
    
    return [letter + l for letter in letters for l in get_inputs(letters, n-1)]

In [13]:
inputs = get_inputs(['a', 'b', 'c', 'd', 'e'], 8)

In [14]:
inputs[:10] # print out the first 10

['aaaaaaaa',
 'aaaaaaab',
 'aaaaaaac',
 'aaaaaaad',
 'aaaaaaae',
 'aaaaaaba',
 'aaaaaabb',
 'aaaaaabc',
 'aaaaaabd',
 'aaaaaabe']

In [15]:
scores = list(map(objective_fun, inputs))
np.max(scores)

5

5 words using only 5 letters! Let's see what string led to this:

In [16]:
best_s = inputs[np.argmax(scores)]
best_s

'aabedadd'

And the words:

In [17]:
english_words_in_string(best_s)

{'a', 'add', 'be', 'bed', 'dad'}

Can you imagine coming up with this by hand?

We can also check if the solution is unique:

In [18]:
np.sort(scores)

array([0, 0, 0, ..., 5, 5, 5])

Looks like the solution is not unique (we see several 5's). 

#### Another example: shortest path in a graph

- Last week, we talked about finding the shortest path between two nodes $A$ and $B$ in a graph.
- This is itself a discrete optimization problem.

1. Possible inputs: all possible paths from $A$ to $B$.
2. Objective function: the length of the path.
3. We are minimizing.

#### Solving an optimization problem

- There are many ways to go about solving an optimization problem. 
- In general, one possibility is _brute force_ in which all possible inputs are considered.
- That's the approach we just took in our toy example above.
- But this is often infeasible because it would be too slow.
  - Think about $26^8$ strings.
  - Think about all possible paths between two nodes in the Facebook graph!
- So, the hard part about optimization is often in finding clever ways to solve the problem faster, that still give the correct solution.
  - E.g. breadth-first search to find the shortest distance in a graph.
  - For the English words problem, there is nothing better than brute force. But there often is.
- The other hard part can be coverting a conceptual idea into the right mathematical form as an optimization problem. This is called _formulating_ the problem.

## Constraints (5 min)

- A popular way of expressing the space of possible inputs is to put _constraints_ on the possibilities.
- Example: all length-8 strings such that each letter appears at most twice.
  - `aabbccdd` is valid, `abcdefgh` is valid, but `aaabbbcc` is _not_ valid ("violates the constraint").
- You could say that this is just a change in the possible inputs, and it is.
  - But, because of how these problems are solved under the hood, we often think of constraints as a separate thing.
  

(Updated) To specify an optimization problem, we need to specify a few things:

1. A specification of the space of possible inputs.
2. An _objective function_ which takes an input and computes a score. 
4. (Optional) A set of _constraints_, which take an input and return true/false.
3. Are we maximizing or minimizing?

In [19]:
def characters_appear_at_most_twice(s, n=2):
    """
    Checks whether all characters appear at most n times in s.
    If any character appears n+1 or more times, returns False.
    
    Parameters
    ----------
    s : str
        The input string
    n : int, optional
        The maximum number of times a character is allowed to appear (default: 2)
        
    Returns
    -------
    bool
        Whether or not all characters appear at most twice
        
    Examples
    --------
    >>> characters_appear_at_most_twice("aabbccdef")
    True
    >>> characters_appear_at_most_twice("aaa")
    False
    >>> characters_appear_at_most_twice("aaa", 5)
    True
    """
    
    return max(s.count(char) for char in s) <= n

In [20]:
assert characters_appear_at_most_twice("aabbccdef")
assert not characters_appear_at_most_twice("aaa")
assert characters_appear_at_most_twice("aaa", 5)

Now we can solve the _constrained optimization problem_:

In [21]:
inputs[:10] # the ones from before

['aaaaaaaa',
 'aaaaaaab',
 'aaaaaaac',
 'aaaaaaad',
 'aaaaaaae',
 'aaaaaaba',
 'aaaaaabb',
 'aaaaaabc',
 'aaaaaabd',
 'aaaaaabe']

In [22]:
constrained_inputs = [s for s in inputs if characters_appear_at_most_twice(s)]

In [23]:
len(inputs)

390625

In [24]:
len(constrained_inputs)

63000

In [25]:
constrained_inputs[:10]

['aabbccdd',
 'aabbccde',
 'aabbcced',
 'aabbccee',
 'aabbcdcd',
 'aabbcdce',
 'aabbcddc',
 'aabbcdde',
 'aabbcdec',
 'aabbcded']

In [26]:
constrained_scores = list(map(objective_fun, constrained_inputs))
np.max(constrained_scores)

4

In [27]:
constrained_best_s = constrained_inputs[np.argmax(constrained_scores)]
constrained_best_s

'abadbedc'

In [28]:
english_words_in_string(constrained_best_s)

{'a', 'bad', 'be', 'bed'}

Some things to note:

- The constrained optimum can be equal to, or worse than, the unconstrained optimum.
  - In this case it was worse - we got 4 words instead of 5 by not allowing repeats. 
- The solution to the constrained problem might be completely different from the unconstrained.
  - For example, here we don't have just 4 of the original 5 words - we have a new word, "bad'.
- It is possible to have multiple constraints.
  - In this case we only had one constraint.
  - In general, for an input to be valid, _all_ the constraints must be satisfied. 
  - For example: each letter appears at most twice AND the string is a palindrome AND it does not contain the substring "ab" (3 constraints).

## TA assignment example: introduction (5 min)

Consider the problem of assigning TAs to labs in MDS.

In [3]:
TAs_df = pd.read_csv("data/TA_apps.csv").set_index("name")
str_to_list = lambda s: list(map(int,s.split(",")))
TAs_df["can_teach"   ] = TAs_df["can_teach"  ].apply(str_to_list)
TAs_df["enthusiastic"] = TAs_df["enthusiastic"].apply(str_to_list)
TAs_df

,availability,R_proficiency,python_proficiency,can_teach,enthusiastic
name,,,,,
Alice,MT,competent,expert,"[542, 551, 552]","[511, 571, 513]"
Bob,MTWR,competent,none,"[551, 523, 531, 552, 561]","[551, 523, 561]"
Carol,M,expert,expert,"[542, 551, 523, 552, 571]","[551, 552, 571]"
Chuck,MW,expert,beginner,"[511, 551, 531, 561]","[521, 542, 523]"
Dan,MTR,expert,expert,"[511, 521, 512, 552, 532]","[521, 512, 552]"
Erin,R,expert,expert,"[512, 532, 513]","[512, 513]"
Faith,MTWR,competent,expert,"[511, 521, 523, 531, 512]","[521, 512, 513]"
Grace,M,beginner,expert,"[521, 523, 512]",[511]
Heidi,TWR,expert,competent,[531],"[511, 551, 523, 552, 561]"


In [4]:
TAs = TAs_df.index.values.tolist()

In [5]:
courses_df = pd.read_csv("data/MDS_courses_term_1.csv").set_index("course_number")
courses_df

,course_title,slug,block,primary_lang,lab_days
course_number,,,,,
511,Programming for Data Science,prog-dsci,1,both,WR
521,Computing Platforms for Data Science,platforms-dsci,1,both,MT
542,Communication and Argumentation,comm-arg,1,none,WR
551,Descriptive Statistics and Probability for Dat...,stat-prob-dsci,1,none,MT
512,Algorithms and Data Structures,alg-data-struct,2,python,WR
523,Data Wrangling,data-wrangling,2,R,WR
531,Data Visualization I,viz-1,2,both,MT
552,Statistical Inference and Computation I,stat-inf-1,2,R,MT
513,Databases and Data Retrieval,database-data-retr,3,both,WR


In [6]:
courses = courses_df.index.values.tolist()

In [7]:
blocks = set(courses_df["block"])

Goal: match TAs to courses so that our staffing needs are covered in the best way possible.

Ok, let's make this more concrete. Here are the **constraints**:

- Each course should be assigned exactly 2 TAs.
- A TA can only cover one course at a time (i.e., in a given block).
- A TA can only be assigned to a course they have listed as "can teach" or "enthusiastic to teach".
- To cover a course, the TA must be available for one of the two lab days (for simplicity - this does not quite guarantee lab coverage).

And here is the **objective**:

- We want to maximize the number of assigned courses that TAs are enthusiastic about.

## (Time permitting) Let's try solving this by hand (10 min)

(on the whiteboard)

If current time < 10:05am, then we have time for this. Else skip.

## Break (5 min)

## Formulating an optimization problem (35 min)

#### Are we maximizing or minimizing?

We are maximizing.

In [57]:
prob = pulp.LpProblem("TA assignments", pulp.LpMaximize)

#### Setting up the decision variables

- For this problem, what are our _decision variables_? 
- Turns out such problems are best handled with _binary variables_.

Let $x_{ij}$ be $1$ if TA $i$ is assigned to course $j$, and 0 otherwise.

In [58]:
x_df = pd.DataFrame(data=(np.random.rand(len(TAs),len(courses))<.2).astype(int), index=TAs, columns=courses)
x_df

,511,521,542,551,512,523,531,552,513,561,571,532
Alice,0,0,0,0,0,0,0,0,1,0,0,1
Bob,1,0,0,0,0,0,1,0,0,0,1,1
Carol,0,0,0,0,1,0,0,0,1,0,1,0
Chuck,0,0,0,0,1,1,1,0,0,1,0,1
Dan,0,0,0,0,1,0,0,0,0,0,1,0
Erin,0,1,0,1,0,0,0,1,1,0,0,0
Faith,1,0,0,0,0,0,0,1,0,0,0,0
Grace,1,0,1,0,0,0,0,0,0,0,0,0
Heidi,0,0,0,0,0,1,0,0,0,0,0,0
Ivan,0,0,1,0,1,0,0,0,0,0,0,0


**NOTE**: this table is not used for solving the problem below - it is only for illustration!

In [59]:
x = pulp.LpVariable.dicts("x", (TAs, courses), 0, 1, pulp.LpInteger) 

How many possible tables are there?

In [60]:
num_entries = len(TAs) * len(courses)
num_entries

120

In [61]:
2**num_entries

1329227995784915872903807060280344576

**(Optional)** Is our formulation "efficient" ?

- With $12$ courses and $2$ TAs per course, there are 24 "spots" and $10$ possible TAs per spot:

In [62]:
TAS_PER_COURSE = 2

len(TAs)**(len(courses)*TAS_PER_COURSE)

1000000000000000000000000

- Our formulation with binary variables has _more_ configurations because it doesn't have the "2 TAs per course" constraint hardwired into it. 
- This seems "redundant" but is actually the way to go because it allows us to use PuLP. 
- Even though we generate a situation with so many more possibilities, this is still the best approach.

(end optional)

#### Setting up the constraints

We now have to change our English constraints into constraints on $x$:

_Each course should be assigned exactly 2 TAs._
  - For all courses $j$, we require $\sum_i x_{ij} = 2$
  - That is, the sum of TAs for DSCI 511 equals 2, the sum of TAs for DSCI 521 equals 2, ...


In [63]:
for course in courses:
    prob += ( pulp.lpSum(x[TA][course] for TA in TAs) == TAS_PER_COURSE )

In [64]:
print(prob)

TA assignments:
MAXIMIZE
None
SUBJECT TO
_C1: x_Alice_511 + x_Bob_511 + x_Carol_511 + x_Chuck_511 + x_Dan_511
 + x_Erin_511 + x_Faith_511 + x_Grace_511 + x_Heidi_511 + x_Ivan_511 = 2

_C2: x_Alice_521 + x_Bob_521 + x_Carol_521 + x_Chuck_521 + x_Dan_521
 + x_Erin_521 + x_Faith_521 + x_Grace_521 + x_Heidi_521 + x_Ivan_521 = 2

_C3: x_Alice_542 + x_Bob_542 + x_Carol_542 + x_Chuck_542 + x_Dan_542
 + x_Erin_542 + x_Faith_542 + x_Grace_542 + x_Heidi_542 + x_Ivan_542 = 2

_C4: x_Alice_551 + x_Bob_551 + x_Carol_551 + x_Chuck_551 + x_Dan_551
 + x_Erin_551 + x_Faith_551 + x_Grace_551 + x_Heidi_551 + x_Ivan_551 = 2

_C5: x_Alice_512 + x_Bob_512 + x_Carol_512 + x_Chuck_512 + x_Dan_512
 + x_Erin_512 + x_Faith_512 + x_Grace_512 + x_Heidi_512 + x_Ivan_512 = 2

_C6: x_Alice_523 + x_Bob_523 + x_Carol_523 + x_Chuck_523 + x_Dan_523
 + x_Erin_523 + x_Faith_523 + x_Grace_523 + x_Heidi_523 + x_Ivan_523 = 2

_C7: x_Alice_531 + x_Bob_531 + x_Carol_531 + x_Chuck_531 + x_Dan_531
 + x_Erin_531 + x_Faith_531 + x_

- Above: in PuLP we use the `+=` syntax to add a new constraint to the optimization problem. 
  - No code to solve the optimization problem is actually been run here.
  - Rather, PuLP is using the syntax of python to allow the user to specify the problem over multiple lines of code.
  - You will see this "language within a language" pattern again in DSCI 572 and DSCI 553, and maybe other places.
- Also, you can think of `pulp.lpSum` as the same as `sum` or `np.sum`.
  - However, it's not doing the summing now, like we usually do with python code.
  - It's just a way of communicating the math of the optimization problem. 

<br>

_A TA can only cover one course at a time (i.e., in a given block)._
  - For all TAs $i$, for all blocks $B$, we require $\sum_{j\in B} x_{ij} \leq 1$
  - That is, the sum of Alice's Block 1 courses is at most 1, the sum of Bob's Block 1 courses is at most 1, ...

In [65]:
for TA in TAs:
    for block in blocks:
        courses_in_block = courses_df.query("block == @block").index # DSCI 523 FTW!
        prob += pulp.lpSum(x[TA][course] for course in courses_in_block) <= 1

<br> 

_A TA can only be assigned to a course they have listed as "can teach" or "enthusiastic to teach"._
  - For all $i,j$ such that canteach$(i,j)$ is false and enthusiastic$(i,j)$ is false, $x_{ij}=0$
  - That is, Bob cannot teach 511 because canteach(Bob,511) is false and enthusiastic(Bob,511) is false, ...
 


In [66]:
for TA in TAs:
    for course in courses:
        if course not in TAs_df.loc[TA,"can_teach"] and course not in TAs_df.loc[TA,"enthusiastic"]:
            prob += ( x[TA][course] == 0 )

<br> 

_To cover a course, the TA must be available for one of the two lab days._
  - For all $i,j$ such that available$(i,\text{day1}(j))$ is false and available$(i,\text{day2}(j))$ is false, $x_{ij}=0$
  - That is, Carol cannot teach DSCI 542 because available(Carol,W) is false and available(Carol,Th) is false
    - And day1(542) is W, day2(542) is Th.
  

In [67]:
for TA in TAs:
    for course in courses:
        if courses_df.loc[course,"lab_days"][0] not in TAs_df.loc[TA]["availability"] and \
           courses_df.loc[course,"lab_days"][1] not in TAs_df.loc[TA]["availability"]:
            prob += x[TA][course] == 0

#### Setting up the objective

- _We want to maximize the number of assigned courses that TAs are enthusiastic about._
- Below we show 3 equivalent ways of specifying this objective in PuLP.

1st approach:

- We want to maximize the following objective: for all $i,j$ such that enthusiastic$(i,j)$ is true, $\sum_{ij} x_{ij}$
- We first do this with a gnarly generator comprehension:

In [20]:
prob += pulp.lpSum(x[TA][course] for TA in TAs for course in courses if course in TAs_df.loc[TA,"enthusiastic"])

- Above: to specify the objective in PuLP, we again use `+=` but this time without any boolean operator on the right side.

2nd approach:

- Here's an alternative version that is totally eqivalent but should be more readable:

In [58]:
objective_terms = list()
for TA in TAs:
    for course in courses:
        if course in TAs_df.loc[TA,"enthusiastic"]:
            objective_terms.append(x[TA][course])
prob += pulp.lpSum(objective_terms)

3rd approach:

- We can also think of the math a bit differently: maximize $\sum_{ij} \text{enthusiastic}(i,j)x_{ij}$

In [48]:
objective_terms = list()
for TA in TAs:
    for course in courses:
        enthusiastic = course in TAs_df.loc[TA,"enthusiastic"] # True/1 if enthusiastic, False/0 otherwise
        objective_terms.append(enthusiastic * x[TA][course])
prob += pulp.lpSum(objective_terms)

- Above: you may need to think about this for a few minutes.
- Because $x_{ij}$ are binary and `enthusiastic` is also binary, the product is 1 only when both are 1.
  - So we're counting up the number of cases where the TA is enthusastic _and_ the TA is assigned to that course.


- Approaches 1 & 2 work because the enthusiasm function is binary (yes/1 or no/0).
- But this can generalize to any sort of numbers. You could have, for example, TAs rate their enthusiasm from 1 to 10.
  - In that case, only approach 3 would work; you'd multiply the enthusiasm score by $x_{ij}$.
  - In the lab you'll need something like approach 3 for the same reason.

#### Solving the problem

In [49]:
prob.solve()

1

In [50]:
pulp.LpStatus[prob.status]

'Optimal'

In [51]:
print("We have %d enthusiastic courses out of a possible %d." % 
      (pulp.value(prob.objective), len(courses)*TAS_PER_COURSE))

We have 16 enthusiastic courses out of a possible 24.


#### Printing the results

In [52]:
out_df_by_TA = pd.DataFrame("", index=TAs, columns=blocks)
for TA in TAs:
    for course in courses:
        if pulp.value(x[TA][course]) == 1:
            out_df_by_TA.loc[TA, courses_df.loc[course, "block"]] = course
out_df_by_TA

,1,2,3
Alice,,552,571
Bob,551,531,561
Carol,551,552,571
Chuck,542,523,
Dan,511,512,532
Erin,,512,513
Faith,521,531,513
Grace,521,,
Heidi,511,523,561
Ivan,542,,532


In [53]:
tas_by_course = defaultdict(list)
for course in courses:
    for TA in TAs:
        if pulp.value(x[TA][course]) == 1:
            tas_by_course[course].append(TA)
out_df_by_course = pd.DataFrame.from_dict(data=tas_by_course, columns=["TA 1", "TA 2"], orient="index")
out_df_by_course

,TA 1,TA 2
511,Dan,Heidi
521,Faith,Grace
542,Chuck,Ivan
551,Bob,Carol
512,Dan,Erin
523,Chuck,Heidi
531,Bob,Faith
552,Alice,Carol
513,Erin,Faith
561,Bob,Heidi


In [54]:
# Enthusiastic courses
for course in courses:
    for TA in TAs:
        if pulp.value(x[TA][course]) == 1 and course in TAs_df.loc[TA]["enthusiastic"]:
            print("%-7s is enthusiastic about DSCI %d!" % (TA, course))

Heidi   is enthusiastic about DSCI 511!
Faith   is enthusiastic about DSCI 521!
Chuck   is enthusiastic about DSCI 542!
Bob     is enthusiastic about DSCI 551!
Carol   is enthusiastic about DSCI 551!
Dan     is enthusiastic about DSCI 512!
Erin    is enthusiastic about DSCI 512!
Chuck   is enthusiastic about DSCI 523!
Heidi   is enthusiastic about DSCI 523!
Carol   is enthusiastic about DSCI 552!
Erin    is enthusiastic about DSCI 513!
Faith   is enthusiastic about DSCI 513!
Bob     is enthusiastic about DSCI 561!
Heidi   is enthusiastic about DSCI 561!
Alice   is enthusiastic about DSCI 571!
Carol   is enthusiastic about DSCI 571!


#### On your own time

- Try playing around with the code above when you remove constraints.
- This will give you a good feeling for the optimization problem.
- With fewer constraints you'll get a better objective value, but the solutions won't make sense.
- Often when looking at the solution you'll realize you forgot a constraint, like a TA doing multiple courses in the same block.

#### So how did this work?

- Earlier we solved the "English words in a string" problem with brute force.
- What would that look like here?
- Well, how big is $x$?

In [ ]:
len(TAs)

In [ ]:
len(courses)

In [ ]:
len(TAs) * len(courses)

- We have 132 binary variables. 
- So, there are $2^{132}$ possible configurations of our binary variables.

In [ ]:
2**132

- Brute force is completely out of the question here.
- What did PuLP do to make it so fast?
- It took advantage of the problem being _linear_.
- For example, we have a constraint that a TA can only cover one course per block, which we encoded as $\sum_j x_{ij} \leq 1$
- So if PuLP is considering a solution where $\sum_j x_{ij}=2$ for some TA $i$, it can discard _all_ possible solutions like this, regardless of the configurations of _all other TAs_.
  - That is a HUGE time saving.
  - Furthermore it can discard all possible solutions with _more_ courses for that TA (i.e. 3 or 4 courses in a block). 
- This type of reasoning wouldn't have worked for the English words example because the solver doesn't "know" the constraints. For example if `aaabbbbb` is not allowed because you can't have more than 2 of a letter, it has no way to know that `aaabbbbd` is also violating the constraint - it can only find out whether the constraint is `True` or `False` for a given input without knowing the _structure_.
- The above discussion pertains to exploiting the linearity of the constraints; there are similar gains to be had from the linearity of the objective.

#### (optional) Regular (non-integer) linear programming

- FYI that the term "linear programing" (LP) usually refers to the case where the decision variables ($x$) are continuous; what we did with discrete $x$ is called integer linear programming (ILP).
  - (You can also have a mix of continuous and discrete variables.)
- PuLP can solve these types of problems as well.
  - The algorithms under the hood are very different, but from your perspective the interface with PuLP is the same whether the variables are continuous or discrete.
- (optional) FYI, discrete problems are generally harder to solve. They have exponential worst-case time though in practice it's fine. Continuous LP problems have better big O bounds. 

#### (optional) What we actually do for MDS TAs:

- All of the above, and...
- Match TA programming language proficiency with courses.
- Respect TA preferences on max or min number of courses (e.g. 1-2 courses, exactly 3 courses, etc).
  - This means you no longer have 3 separate problems for the 3 blocks.
- Allow the script to under/over staff if necessary (big term in the objective).
- And more!

## Types of discrete optimization problems (5 min)

- We tend to classify discrete optimization problems into different types, based on the tricks we can use to solve them.
- For example, some tough optimization problems are said to be [NP-complete](https://en.wikipedia.org/wiki/NP-completeness). We won't go into detail here, but it's related to the idea that you have to use brute force.
- Other optimization problems are called _linear_. Discrete optimization problems that are linear are called [integer linear programs](https://en.wikipedia.org/wiki/Integer_programming). 
  - There is a technical definition of linear optimization problems, but in short the objective and constraints must be linear functions of the inputs. 
  - In Python, we can conveniently solve linear optimization problems using the [PuLP](https://pythonhosted.org/PuLP/) package.
  - The way these solvers work is outside the scope of MDS.
- In general, if your optimization problem falls into a more restrictive class of problems, there are probably tools to solve them faster. 
- Next class we'll talk about another class of problems that can be solved fast, using "dynamic programming".

## T/F questions (10 min)

1. A solver will violate the constraints if this results in huge gains for the objective function.
2. Some discrete optimization problems have multiple solutions.
3. Some discrete optimization problems have no solutions at all.
4. PuLP can be use to solve any optimization problem, but it may be very slow for some problems.